# Global-Wheat-Detection: EfficientDet Inference

This is the inference notebook to my EfficientDet pipeline. It is based on Alex Shonenkov's [notebook](https://www.kaggle.com/shonenkov/inference-efficientdet), since before this I have never used EfficientDet and the documentation of the PyTorch implementation I am using is very spotty. 

## Setup

### System Detection

This is to make sure the notebook works on both my own windows workstation as well as on the Kaggle

In [ ]:
import os
system = os.name
if system == 'posix':
    kaggle = True
    windows = False
    print('running on kaggle')
elif system == 'nt':
    kaggle = False
    windows = True
    print('running on windows')
else:
    print('unknown system')

### Imports

In [ ]:
if windows:
    import os
    import numpy as np
    import pandas as pd
    from glob import glob
    import re
    import cv2
    import torch
    from torch.utils.data import DataLoader
    from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
    from effdet.efficientdet import HeadNet
    import gc
    import matplotlib.pyplot as plt

In [ ]:
if kaggle:
    !pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
#     !pip install --no-deps '../input/effdetgithub/'
    !pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
if kaggle:
    import sys
    sys.path.insert(0, "../input/timm-efficientdet-pytorch")
    sys.path.insert(0, "../input/omegaconf")
    sys.path.insert(0, "../input/weightedboxesfusion")

    from ensemble_boxes import *
    import torch
    import numpy as np
    import pandas as pd
    from glob import glob
    from torch.utils.data import Dataset,DataLoader
    import albumentations as A
    from albumentations.pytorch.transforms import ToTensorV2
    import cv2
    import gc
    from matplotlib import pyplot as plt
    from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
    from effdet.efficientdet import HeadNet
    import re

### List of all test files

In [ ]:
path = f'../input/global-wheat-detection/train'
# path = f'../input/global-wheat-detection/test'
jpg_paths = glob(f'{path}/*.jpg')
jpgfilenames = [re.compile(r'([\w\_\(\)]*)\.jpg').search(jpgpath).group(1) for jpgpath in jpg_paths]

### Configuration

In [ ]:
class configuration():
    scale = 1
    batch_size = 4
    if windows:
        cuda_device = 1
    if kaggle:
        cuda_device = 0
    if kaggle:
        gt = False
        visualize = False
        save = False
    if windows:
        gt = True
        visualize = True
        save = True

### DatasetRetriever

In [ ]:
# Retriever for testing data
class DatasetRetriever():
    
    def __init__(self, image_ids):
        super().__init__()
        self.image_ids = image_ids
        self.scale = configuration.scale
    def __getitem__(self, index: int):
        filename = self.image_ids[index]
        image = f'{path}/{filename}.jpg'
        image = cv2.imread(image, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        #resize the image
        w_image = int(image.shape[1] * self.scale)
        h_image = int(image.shape[0] * self.scale)
#         dim = (w_image, h_image)
        dim = (1024, 1024)
        image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        image = torch.tensor(image).permute(2,0,1)
        
        return image, filename
    
    def __len__(self) -> int:
        return len(self.image_ids)
    
dr = DatasetRetriever(jpgfilenames)
dr.__len__()

In [ ]:
if configuration.gt == True:
    nr_files = len(jpgfilenames)
    class DatasetRetriever1():
        def __init__(self, df, image_ids):
            super().__init__()
            self.image_ids = image_ids
            self.df = df
        def __getitem__(self, index:int):
            #load the image
            filename = self.image_ids[index][:-2]
            flavor = self.image_ids[index][-2:]
            filename = self.image_ids[index]
            image = f'{path}/{filename}.jpg'
            image = cv2.imread(image, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
            image /= 255.0
            #resize the image
            dim = (1024, 1024)
            image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            image = torch.tensor(image).permute(2,0,1)
            if filename in labels.image_id.values:
                boxes = labels.loc[labels.image_id == filename,['x_min', 'y_min', 'x_max', 'y_max']]
                boxes = torch.tensor(boxes.values)
                boxes = boxes[:,[1,0,3,2]]
                boxes = boxes.int()
                lbls = torch.tensor(np.ones(boxes.shape[0], dtype = 'int32'))
            else:
                boxes = torch.tensor([])
                lbls = torch.tensor([])
            return image, dict(boxes = boxes, labels = lbls, image_id = torch.tensor([index])), filename
        def __len__(self):
            return len(self.image_ids)

    labels = pd.read_csv('labels.csv')
    dataset1 = DatasetRetriever1(df = labels, image_ids = jpgfilenames)

In [ ]:
dataset = DatasetRetriever(image_ids = jpgfilenames)

### Collate

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

### Data Loader 

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=configuration.batch_size,
                                          pin_memory = False,
                                          drop_last = False,
                                          shuffle = False,
                                          collate_fn = collate_fn)

### Load the Network

In [ ]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config = config, pretrained_backbone = False)
    config.num_classes = 1
    config.image_size = 1024 * configuration.scale
    net.class_net = HeadNet(config, num_outputs = config.num_classes,
                            norm_kwargs = dict(eps = .001, momentum = .01))
    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    
    
    del checkpoint
    gc.collect()
    if windows:
        net = DetBenchPredict(net, config)
    if kaggle:
        net = DetBenchEval(net, config)
    return net.cuda(configuration.cuda_device)


In [ ]:
# chkpntpth = f'../../train/working/effdet-cutmix-augmix/last-checkpoint.bin'
# chkpntpth = sorted(glob(f'../../train/working/effdet-cutmix-augmix/best-checkpoint-*epoch.bin'))[-1]
if windows:
    chkpntpth = sorted(glob(f'../../train/working/old/effdet-experimental/best-checkpoint-*epoch.bin'))[-1]
if kaggle:
    chkpntpth = f'../input/effdetweights/last-checkpoint.bin'
# chkpntpth = f'../input/effdet-cutmix-augmix/last-checkpoint.bin'
net = load_net(chkpntpth)

### Make Predictions

In [ ]:
def make_predictions(images, image_ids, score_threshold = 0.4):
    images = torch.stack(images).cuda(configuration.cuda_device).float()
    predictions = {'image_id':[],'conf': [],'boxes':[]}#, 'bad_conf':[], 'badboxes':[]}
    
    batch_size = images.shape[0]
    images = images.cuda(configuration.cuda_device).float()
    target_res = {}
    target_res['img_scale'] = torch.tensor([1.0] * batch_size, dtype = torch.float32).cuda(configuration.cuda_device)
    target_res['img_size'] = torch.tensor([images[0].shape[-2:]] * batch_size, dtype=torch.float32).cuda(configuration.cuda_device)
    with torch.no_grad():
        if windows:
            det = net(images, target_res['img_scale'], target_res['img_size'])
        if kaggle:
            det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
            
        for i in range(images.shape[0]):
            # the first four columns are the boundaries of the boxes and the last 2 are the 
            boxes = det[i].detach().cpu().numpy()[:,:4]
            scores = det[i].detach().cpu().numpy()[:,4]
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            scores = scores[indexes]
            if boxes.shape[0] < 5:
                score_threshold = 0.6
            else:
                score_threshold = np.average(scores) - 0.2
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            scores = scores[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            #predictions.append({'boxes': boxes[indexes],
             #                 'scores': scores[indexes]})
            predictions['boxes'].append(boxes)
            predictions['conf'].append(scores)
            predictions['image_id'].append(image_ids[i])
#             predictions['bad_conf'].append()
            
    return predictions
#     with torch.no_grad():
        

### Visualize Predictions

In [ ]:
if configuration.visualize == True:
    m = 0
    new_imagelist = []
    inf_folder = f'./inference'
    if configuration.save & (not os.path.exists(inf_folder)):
                os.makedirs(inf_folder)
    for j, (images, image_ids) in enumerate(data_loader):
        predictions = make_predictions(images, image_ids)
        df = pd.DataFrame(predictions)
        for i in range(len(image_ids)):
            ## plotting the inference
            image_id = image_ids[i]
            new_imagelist.append(image_id)
            fig, ax = plt.subplots(1, 1, figsize=(16, 16))
            bx1 = df.iloc[i].boxes
            conf1 = df.iloc[i].conf
            sample = images[i].permute(1,2,0).cpu().numpy()
            k = 0
            for box in bx1:
                cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)
                plt.text(box[2],box[1],f'{conf1[k]:.3f}', color = 'red', fontsize = 18)
                k+=1
            #Plotting the Ground Truth
            if configuration.gt == True:
                image1, target1, image_id1 = dataset1[m]
                boxes1 = target1['boxes'].cpu().numpy().astype(np.int32)
                for box1 in boxes1:
                    cv2.rectangle(sample, (box1[1], box1[0]), (box1[3],  box1[2]), (0, 1, 0), 2)

            #Showing and saving the picture
            ax.set_axis_off()
            ax.imshow(sample)
            if windows & configuration.save:
                plt.savefig(f'{inf_folder}/{image_ids[i]}.jpeg', transparent=True, bbox_inches = 'tight',
                            facecolor = 'k',pad_inches = 0)
            plt.show()
            m+=1

In [ ]:
submission = pd.DataFrame(columns = ['image_id', 'PredictionString'])
cols = ['conf', 'px_min', 'py_min', 'px_max', 'py_max']

for j, (images, image_ids) in enumerate(data_loader):
    predictions = make_predictions(images, image_ids)
    df = pd.DataFrame(predictions)
#     print('dataframe',df)
    for i in range(len(image_ids)):
        row = df.iloc[i]
#         print('row',row)
#         print(row)
        filename = row.image_id
        dataset = pd.DataFrame(columns = cols)
        if len(row.conf) > 0:
            for k in range(len(row.conf)):
                row_df = pd.DataFrame([[row.conf[k],row.boxes[k][0],row.boxes[k][1],row.boxes[k][2],row.boxes[k][3]]], 
                                      columns = cols)
                dataset = pd.concat([dataset, row_df], axis = 0, ignore_index = True)
            dataset['width'] = 1024 * configuration.scale
            dataset['height'] = 1024 * configuration.scale
            dataset['px_box'] = dataset['px_max'] - dataset['px_min']
            dataset['py_box'] = dataset['py_max'] - dataset['py_min']
            dataset[['px_min', 'py_min', 'px_box', 'py_box']] = dataset[['px_min', 'py_min', 'px_box', 'py_box']].round().astype('int')
            dataset.loc[dataset.width < dataset.px_min,'px_min'] = dataset.width -1
            dataset.loc[dataset.height < dataset.py_min,'py_min'] = dataset.height -1
            dataset.loc[1 > dataset.px_box,'px_box'] = 1
            dataset.loc[1 > dataset.py_box,'py_box'] = 1
            dataset.loc[0 > dataset.px_min,'px_min'] = 0
            dataset.loc[0 > dataset.py_min,'py_min'] = 0
            dataset.loc[dataset['px_min'] + dataset['px_box'] > dataset['width'], 'px_box'] = dataset['width'] - dataset['px_min'] - 1
            dataset.loc[dataset['py_min'] + dataset['py_box'] > dataset['height'], 'py_box'] = dataset['height'] - dataset['py_min'] - 1
            submission_cols = ['conf', 'px_min', 'py_min', 'px_box', 'py_box']
            shape = dataset[submission_cols].shape
            row1 = dataset[submission_cols].values.reshape(1,shape[0]*shape[1])
            str_array = np.array2string(row1[0], formatter = {'float_kind': lambda x: '%.5f' % x}, max_line_width = 9e3)
            str_array = re.compile(r'\[([\d\s\.]+)\]').search(str_array).group(1)
            vals = pd.DataFrame([[filename, str_array]], columns =['image_id', 'PredictionString'])
        else:
            vals = pd.DataFrame([[filename, '']], columns =['image_id', 'PredictionString'])
        submission = pd.concat([submission,vals], axis = 0, ignore_index = True)
        submission.PredictionString = submission.PredictionString.str.replace(r'(\.0+)', '').str.replace('\s+',' ')

submission.to_csv('submission.csv', index = False, sep = ',')
submission